In [72]:
import pandas as pd
import glob
import os
from itertools import chain
import numpy as np
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

#load data
good_df = pd.read_csv('good_games.csv')

#put data into dataframe
good_df = pd.DataFrame(good_df)
#print for debugging
good_df

,Name,AppID,Short Description,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Review Text (First Review),Playtime Forever (First Review),Playtime Last Two Weeks (First Review),Playtime at Review (First Review),Timestamp Created (First Review),Timestamp Updated (First Review),Voted Up (First Review),Votes Up (First Review),Genres
0,Necromunda: Hired Gun,1222370,"Become a Hired Gun. The money’s good, the dog’...",6,Mostly Positive,2264,921,3185,71.083203,"A clunky arcade style fps. Its an unfinished, ...",71.0,0.0,71.0,2023-11-03 06:11:21,2023-11-03 06:29:40,0.0,15.0,"Action, Adventure"
1,The Soul Stone War,1222400,"Become the Wielder of Soul Stone, an ancient a...",6,Mostly Positive,55,22,77,71.428571,"First and Foremost, once again this book is no...",868.0,0.0,370.0,2020-03-06 06:56:40,2020-03-06 06:56:40,0.0,71.0,"Adventure, Casual, Indie, RPG"
2,A Way Out,1222700,A Way Out - это уникальное коллективное приклю...,8,Very Positive,10875,1564,12439,87.426642,we are not friends anymore but the story was i...,344.0,0.0,344.0,2023-11-11 20:19:51,2023-11-11 20:19:51,1.0,33.0,"Экшены, Приключенческие игры, Инди"
3,FutaDomWorld,1222750,"FutaDomWorld is a dystopian, futanari-femdom d...",8,Very Positive,59,4,63,93.650794,"You crafty LGBTQ+ futa bastards. You made me,...",928.0,0.0,768.0,2023-11-04 20:30:59,2023-11-04 20:30:59,1.0,5.0,"Casual, Indie, Early Access"
4,Umurangi Generation,1223500,Umurangi Generation is a first person photogra...,8,Very Positive,656,48,704,93.181818,"It may be pretentious, but ignore the time lim...",451.0,0.0,216.0,2023-09-08 16:21:07,2023-09-08 16:21:07,1.0,5.0,"Casual, Indie, Simulation"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12789,Filly Astray,2602780,A platforming adventure game with a new take o...,8,Very Positive,52,4,56,92.857143,"TLDR: 6/10. Good effort, extremely cute, but f...",40.0,0.0,40.0,2023-11-02 11:25:12,2023-11-02 11:28:02,1.0,52.0,Action
12790,Buried,2605640,You wake up buried alive. You are running out ...,8,Very Positive,59,14,73,80.821918,GAMEPLAY AND REVIEW https://www.youtube.com/w...,28.0,0.0,28.0,2023-11-16 09:02:07,2023-11-16 09:02:07,1.0,3.0,Indie
12791,Front Office Football Nine,2633170,Put yourself in the Front Office with Front Of...,6,Mostly Positive,40,12,52,76.923077,"Bought it within hours of it coming out, love ...",5566.0,2905.0,82.0,2023-11-02 14:21:02,2023-11-02 14:21:02,0.0,28.0,"Indie, Simulation, Sports"
12792,Lust Bunker [18+],2668170,Welcome to &quot;Lust Bunker&quot; – an enthra...,8,Very Positive,70,10,80,87.500000,The game literally has only 1 decision you can...,97.0,97.0,97.0,2023-11-18 14:02:31,2023-11-18 14:02:31,0.0,64.0,"Casual, RPG, Simulation"


In [73]:
unclean_good_titles = []
counter = 0
while counter < len(good_df):
    unclean_good_titles.append(word_tokenize(good_df.iloc[counter]['Name']))
    counter += 1
print(len(unclean_good_titles))

12794


In [74]:
# create the stop list array, I've been adding to this as I've been going through the output
stops = list(set(stopwords.words('english'))) + [',', '.' , '-', 'however', 'ever' , 'also', '?' , '#', '@' ,'(', ')', "'s", "n't" , '``', "''", 
 "--"]

#lowercase all words
unclean_good_titles = [[word.lower() for word in sentence] for sentence in unclean_good_titles]

#stem words
from nltk.stem import PorterStemmer
ps = PorterStemmer()
unclean_good_titles = [[ps.stem(word) for word in sentence] for sentence in unclean_good_titles]

#lemmatize words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
unclean_good_titles = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in unclean_good_titles]

# get rid of stop words
unclean_good_titles = [[word for word in sentence if word not in stops] for sentence in unclean_good_titles]

# get rid of punctuation
unclean_good_titles = [[word for word in sentence if word not in string.punctuation] for sentence in unclean_good_titles]

# get rid of works that contain numbers
unclean_good_titles = [[word for word in sentence if not any(char.isdigit() for char in word)] for sentence in unclean_good_titles]

# get rid of single character words
unclean_good_titles = [[word for word in sentence if len(word) > 1] for sentence in unclean_good_titles]

# get rid of contractions
unclean_good_titles = [[word for word in sentence if "'" not in word] for sentence in unclean_good_titles]

# get rid of .com
unclean_good_titles = [[word for word in sentence if ".com" not in word] for sentence in unclean_good_titles]

# get rid of non-printable characters
unclean_good_titles = [[word for word in sentence if word.isprintable()] for sentence in unclean_good_titles]

# get rid of emojis by looking for non-ascii characters
unclean_good_titles = [[word for word in sentence if word.encode('ascii', 'ignore').decode('ascii') == word] for sentence in unclean_good_titles]

# get rid of words that contain - / or . or ^
unclean_good_titles = [[word for word in sentence if '-' not in word] for sentence in unclean_good_titles]
unclean_good_titles = [[word for word in sentence if '/' not in word] for sentence in unclean_good_titles]
unclean_good_titles = [[word for word in sentence if '.' not in word] for sentence in unclean_good_titles]
unclean_good_titles = [[word for word in sentence if '^' not in word] for sentence in unclean_good_titles]

#flatten the list
unclean_good_titles = list(chain.from_iterable(unclean_good_titles))
#convert to dataframe
unclean_good_titles = pd.DataFrame(unclean_good_titles)
# unclean_good_titles.to_csv('good_titles_wordcloud_data.csv')
unclean_good_titles

,0
0,necromunda
1,hire
2,gun
3,soul
4,stone
...,...
29827,lust
29828,bunker
29829,suika
29830,watermelon


In [75]:
#create a vocabulary
vocab = list(set(unclean_good_titles[0]))
vocab.sort()
vocab

['+plu',
 'a=b',
 'aaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaa',
 'aaero',
 'aarklash',
 'abalon',
 'abandon',
 'abc',
 'abduct',
 'abe',
 'abod',
 'abomi',
 'abov',
 'abriss',
 'absolut',
 'absolv',
 'absurd',
 'abyss',
 'academagia',
 'academi',
 'academia',
 'accel',
 'acceler',
 'accent',
 'access',
 'accid',
 'accomplish',
 'account',
 'accounting+',
 'accur',
 'accurc',
 'ace',
 'achiev',
 'achil',
 'achra',
 'achron',
 'achtung',
 'acid',
 'acolyt',
 'acorn',
 'acquit',
 'acr',
 'acron',
 'across',
 'act',
 'action',
 'activ',
 'actor',
 'actrais',
 'actress',
 'actual',
 'ad',
 'adaca',
 'adalar',
 'adam',
 'addict',
 'adio',
 'admir',
 'adolf',
 'adom',
 'ador',
 'adrift',
 'adult',
 'advanc',
 'advent',
 'adventur',
 'advers',
 'adversari',
 'advic',
 'advisor',
 'aegi',
 'aeon',
 'aer',
 'aeranni',
 'aerofli',
 'aeterna',
 'aeternam',
 'aether',
 'aetheria',
 'aew',
 'affair',
 'affect',
 'affogato',
 'afghanistan',
 'afraid',
 'africa',
 'afterburn',
 'afterdream',
 'afterimag',
 

In [76]:
#count the number of times each word appears in the vocab from unclean_good_titles
word_counts = []
for word in vocab:
    word_counts.append(unclean_good_titles[0].value_counts()[word])
word_counts = pd.DataFrame(word_counts)
word_counts = word_counts.rename(columns={0: 'count'})
word_counts['word'] = vocab
word_counts = word_counts[['word', 'count']]
word_counts = word_counts.sort_values(by=['count'], ascending=False)
#remove words that appear less than 7 times
word_counts = word_counts[word_counts['count'] > 6]
word_counts.to_csv('good_title_occurrences.csv')
word_counts

,word,count
2308,edit,369
6891,simul,272
2994,game,189
8322,vr,178
8370,war,176
...,...,...
657,bear,7
620,battleground,7
370,arma,7
834,board,7


Repeat for bad data

In [77]:
#load data
bad_df = pd.read_csv('bad_games.csv')
#put data into dataframe
bad_df = pd.DataFrame(bad_df)
#print for debugging
bad_df

,Name,AppID,Short Description,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Review Text (First Review),Playtime Forever (First Review),Playtime Last Two Weeks (First Review),Playtime at Review (First Review),Timestamp Created (First Review),Timestamp Updated (First Review),Voted Up (First Review),Votes Up (First Review),Genres
0,18+ MEMORY,1222470,18+ MEMORY - is a classic memory game where yo...,5,Mixed,11,7,18,61.111111,[h1]At a Glance[/h1] [table] [tr] ...,66.0,0.0,17.0,2020-02-01 08:38:16,2020-02-01 08:38:16,0.0,38.0,"Casual, Indie"
1,Slotracers VR,1222480,Slotracers VR is a full slotracing experience ...,0,3 user reviews,2,1,3,66.666667,Played on HP Reverb G2 (WMR) without issue. T...,1521.0,0.0,79.0,2021-03-06 16:41:00,2021-03-06 16:41:00,1.0,10.0,"Casual, Indie, Racing, Simulation"
2,Gastova: The Witches of Arkana,1222540,Gastova is a lighthearted 2D platforming adven...,0,5 user reviews,5,0,5,100.000000,"Its a fun little adventure, not too hard, not ...",222.0,0.0,221.0,2023-08-10 01:45:34,2023-08-10 01:45:34,1.0,0.0,"Action, Adventure, Indie, RPG, Early Access"
3,Red Ronin,1222570,Red Ronin is a Turn-based but fast paced dash ...,6,Mostly Positive,38,10,48,79.166667,"""Turn Based But Fast Paced"" means, about 1/3 i...",191.0,0.0,191.0,2021-09-28 05:04:07,2021-09-28 05:15:02,0.0,40.0,"Action, Indie, Strategy"
4,Stocksynd House,1222600,Stocksynd House offers a narrative-driven VR e...,7,Positive,34,6,40,85.000000,Some quick notes as I just finished: +Very de...,80.0,0.0,80.0,2020-03-21 04:10:29,2020-03-21 04:10:29,1.0,17.0,"Adventure, Indie"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41353,Tamara In The Forsaken Dungeon,2686290,Tamara In The Forsaken Dungeon is a third-pers...,0,1 user reviews,1,0,1,100.000000,"Clearly an Indy game, but not too bad. Good F-...",67.0,67.0,20.0,2023-11-27 16:25:36,2023-11-27 16:25:36,1.0,3.0,"Adventure, Indie"
41354,Suika Shapes,2686690,Match a variety of colorful shapes in an ever-...,7,Positive,13,0,13,100.000000,This is the Definitive version of Suika. I am ...,376.0,376.0,66.0,2023-11-27 19:03:47,2023-11-27 19:03:47,1.0,9.0,"Action, Casual"
41355,SOS Zombie Survivors,2689600,SOS Zombie Survivors follows a group of surviv...,6,Mostly Positive,10,3,13,76.923077,The game combines traditional survival and wav...,365.0,365.0,302.0,2023-11-28 21:06:10,2023-11-28 21:06:10,1.0,9.0,Action
41356,24H Stories: The Blackout,2690290,"You are a young boy. Tonight, your parents hav...",0,1 user reviews,1,0,1,100.000000,"Great game from an indie developer, really kee...",98.0,98.0,96.0,2023-11-27 18:37:01,2023-11-27 18:37:01,1.0,0.0,Indie


In [78]:
unclean_bad_titles = []
counter = 0
while counter < len(bad_df):
    unclean_bad_titles.append(word_tokenize(str(bad_df.iloc[counter]['Name'])))
    counter += 1
unclean_bad_titles

[['18+', 'MEMORY'],
 ['Slotracers', 'VR'],
 ['Gastova', ':', 'The', 'Witches', 'of', 'Arkana'],
 ['Red', 'Ronin'],
 ['Stocksynd', 'House'],
 ['Model', 'Kit', 'Simulator', 'VR'],
 ['Super', 'Squash'],
 ['VELUCITY'],
 ['Xenovaders'],
 ['Emily', 'Archer', 'and', 'the', 'Curse', 'of', 'Tutankhamun'],
 ['BalloonBoyBob'],
 ['Zombie', 'Towns'],
 ['Castle', 'Torgeath', 'Redux'],
 ['Dish', 'Life', ':', 'The', 'Game'],
 ['War', 'of', 'the', 'Barrenlands'],
 ['Beardy', 'McStrongarm'],
 ['Beyond', 'The', 'Underworld'],
 ['A', 'Player', "'s", 'Heart'],
 ['Mall', 'Craze'],
 ['The', 'Savana'],
 ['Super', 'Rocket', 'Ride'],
 ['ManaTech'],
 ['Tennis', 'World', 'Tour', '2'],
 ['Erra', ':', 'Exordium'],
 ['Secret', 'City', ':', 'Chalk', 'of', 'Fate', 'Collector', "'s", 'Edition'],
 ['Somewhere', ':', 'Sect', 'of', 'Relic'],
 ['My', 'Jigsaw', 'Adventures', '-', 'Roads', 'of', 'Life'],
 ['Fire', 'Escape'],
 ['Inner', 'Worlds'],
 ['Kowi', 'Ishto', ':', 'Battle', 'of', 'Akonoli'],
 ['A', 'Mysterious', 'Galle

In [79]:
# create the stop list array, I've been adding to this as I've been going through the output
stops = list(set(stopwords.words('english'))) + [',', '.' , '-', 'however', 'ever' , 'also', '?' , '#', '@' ,'(', ')', "'s", "n't" , '``', "''", 
 "--"]

#lowercase all words
unclean_bad_titles = [[word.lower() for word in sentence] for sentence in unclean_bad_titles]

#stem words
from nltk.stem import PorterStemmer
ps = PorterStemmer()
unclean_bad_titles = [[ps.stem(word) for word in sentence] for sentence in unclean_bad_titles]

#lemmatize words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
unclean_bad_titles = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in unclean_bad_titles]

# get rid of stop words
unclean_bad_titles = [[word for word in sentence if word not in stops] for sentence in unclean_bad_titles]

# get rid of punctuation
unclean_bad_titles = [[word for word in sentence if word not in string.punctuation] for sentence in unclean_bad_titles]

# get rid of works that contain numbers
unclean_bad_titles = [[word for word in sentence if not any(char.isdigit() for char in word)] for sentence in unclean_bad_titles]

# get rid of single character words
unclean_bad_titles = [[word for word in sentence if len(word) > 1] for sentence in unclean_bad_titles]

# get rid of contractions
unclean_bad_titles = [[word for word in sentence if "'" not in word] for sentence in unclean_bad_titles]

# get rid of .com
unclean_bad_titles = [[word for word in sentence if ".com" not in word] for sentence in unclean_bad_titles]

# get rid of non-printable characters
unclean_bad_titles = [[word for word in sentence if word.isprintable()] for sentence in unclean_bad_titles]

# get rid of emojis by looking for non-ascii characters
unclean_bad_titles = [[word for word in sentence if word.encode('ascii', 'ignore').decode('ascii') == word] for sentence in unclean_bad_titles]

# get rid of words that contain - / or . or ^
unclean_bad_titles = [[word for word in sentence if '-' not in word] for sentence in unclean_bad_titles]
unclean_bad_titles = [[word for word in sentence if '/' not in word] for sentence in unclean_bad_titles]
unclean_bad_titles = [[word for word in sentence if '.' not in word] for sentence in unclean_bad_titles]
unclean_bad_titles = [[word for word in sentence if '^' not in word] for sentence in unclean_bad_titles]

#flatten the list
unclean_bad_titles = list(chain.from_iterable(unclean_bad_titles))
#convert to dataframe
unclean_bad_titles = pd.DataFrame(unclean_bad_titles)
unclean_bad_titles.to_csv('bad_titles_wordcloud_data.csv')
unclean_bad_titles

,0
0,memori
1,slotrac
2,vr
3,gastova
4,witch
...,...
94133,zombi
94134,survivor
94135,stori
94136,blackout


In [80]:
#create a vocabulary
vocab2 = list(set(unclean_bad_titles[0]))
vocab2.sort()
vocab2

['+plu',
 ':epica',
 ':reload',
 '\\raft',
 '_________',
 '_dive',
 'a+',
 'aa',
 'aah',
 'aaron',
 'aartform',
 'aaru',
 'aatral',
 'abacu',
 'abaddon',
 'abalon',
 'abandon',
 'abandonwar',
 'abasralsa',
 'abati',
 'abatron',
 'abberburi',
 'abbey',
 'abbi',
 'abc',
 'abd',
 'abda',
 'abduct',
 'abe',
 'abedot',
 'abelana',
 'abermor',
 'aberoth',
 'aberr',
 'abfens',
 'abh',
 'abha',
 'abi',
 'abiko',
 'abl',
 'ablaz',
 'ableford',
 'ablepsia',
 'abnorm',
 'abo',
 'aboard',
 'abod',
 'abomin',
 'aborigenu',
 'abov',
 'abraca',
 'abracadabrew',
 'abraham',
 'abrakadaboom',
 'abram',
 'abridg',
 'abrix',
 'abscess',
 'absciss',
 'abscond',
 'absenc',
 'absent',
 'absentia',
 'absinth',
 'absinthia',
 'absolut',
 'absorb',
 'absorpt',
 'abst',
 'abstract',
 'abstractank',
 'abstractanoid',
 'absurbia',
 'absurd',
 'absurdika',
 'absurdu',
 'abus',
 'abyss',
 'academi',
 'academia',
 'acan',
 'acanthocera',
 'acaratu',
 'acardshoot',
 'accel',
 'acceler',
 'accender',
 'accept',
 'acces

In [81]:
#count the number of times each word appears in the vocab from unclean_bad_titles
word_counts = []
for word in vocab2:
    word_counts.append(unclean_bad_titles[0].value_counts()[word])
word_counts = pd.DataFrame(word_counts)
word_counts = word_counts.rename(columns={0: 'count'})
word_counts['word'] = vocab2
word_counts = word_counts[['word', 'count']]
word_counts = word_counts.sort_values(by=['count'], ascending=False)
#remove words that appear less than 7 times
word_counts = word_counts[word_counts['count'] > 6]
word_counts.to_csv('bad_title_occurrences.csv')
word_counts

,word,count
19211,vr,1226
5292,edit,844
15879,simul,667
6816,game,660
174,adventur,643
...,...,...
15501,select,7
19457,weekend,7
15469,secur,7
17878,tick,7


Wrangling all the data into useable forms


In [82]:
#import dataframes
good_occurences = pd.read_csv('good_title_occurrences.csv')
bad_occurences = pd.read_csv('bad_title_occurrences.csv')

#sort good_occurences by count
good_occurences = good_occurences.sort_values(by=['count'], ascending=False)
#drop first column from good_occurences
good_occurences = good_occurences.drop(columns=['Unnamed: 0'])
#drop first column from bad_occurences
bad_occurences = bad_occurences.drop(columns=['Unnamed: 0'])

#merge both vocabularies into a set
vocab = list(set(good_occurences['word']) | set(bad_occurences['word']))
vocab

# good_occurences.to_csv('good_occurences.csv')
# bad_occurences.to_csv('bad_occurences.csv')


['flat',
 'wolv',
 'wet',
 'look',
 'mission',
 'protect',
 'resist',
 'lander',
 'pirat',
 'egypt',
 'justic',
 'bundl',
 'seeker',
 'unicorn',
 'cell',
 'dragon',
 'blast',
 'expans',
 'joe',
 'squad',
 'rain',
 'carnag',
 'sg',
 'cool',
 'paintbal',
 'interact',
 'ambit',
 'cow',
 'care',
 'psi',
 'lie',
 'flower',
 'chernobyl',
 'coaster',
 'freddi',
 'soldier',
 'vampir',
 'flex',
 'grisaia',
 'recharg',
 'absolut',
 'zero',
 'remnant',
 'protocol',
 'pocket',
 'european',
 'gener',
 'paranorm',
 'clicker',
 'turbo',
 'union',
 'drifter',
 'loco',
 'neo',
 'corrupt',
 'pandem',
 'freedom',
 'snail',
 'boot',
 'devil',
 'peac',
 'apocalyps',
 'santa',
 'western',
 'roller',
 'social',
 'energi',
 'wave',
 'jam',
 'float',
 'wrath',
 'higurashi',
 'da',
 'racer',
 'signal',
 'supercross',
 'ascend',
 'investig',
 'battlefield',
 'failur',
 'episod',
 'psycho',
 'state',
 'chamber',
 'graviti',
 'mari',
 'arena',
 'nova',
 'pursuit',
 'purgatori',
 'apart',
 'nobunaga',
 'jag',
 'une

Using vocab to build table

In [18]:
#drop the first column in good_occurences
# good_occurences = good_occurences.drop(columns=['word'])
# good_occurences
# bad_occurences

,word,count
0,vr,1226
1,edit,844
2,simul,667
3,game,660
4,adventur,643
...,...,...
2129,select,7
2130,weekend,7
2131,secur,7
2132,tick,7


In [83]:
#create table of the vocab
titles_table = []
titles_table = pd.DataFrame(vocab)
#rename column to word
titles_table = titles_table.rename(columns={0: 'word'})
#add column for good occurences
titles_table['good_occurences'] = 0
#add column for bad occurences
titles_table['bad_occurences'] = 0
#add column for total occurences
titles_table['total_occurences'] = 0
#add column for good game containing word probability
titles_table['good_game_containing_word_probability'] = 0
#add column for bad game containing word probability
titles_table['bad_game_containing_word_probability'] = 0
#add column for good game given word probability
titles_table['good_game_given_word_probability'] = 0
#add column for bad game given word probability
titles_table['bad_game_given_word_probability'] = 0

#map good occurences to table
titles_table = titles_table.merge(good_occurences, on='word', how='left')
#map bad occurences to table
titles_table = titles_table.merge(bad_occurences, on='word', how='left')
#fill NaN values in count_x and count_y with 0
titles_table = titles_table.fillna(0)
#turn count_x and count_y into integers
titles_table['good_occurences'] = titles_table['count_x'].astype(int)
titles_table['bad_occurences'] = titles_table['count_y'].astype(int)
#drop count_x and count_y columns
titles_table = titles_table.drop(columns=['count_x', 'count_y'])

    
# loop through table and add total occurences
for index, row in titles_table.iterrows():
    titles_table.loc[index, 'total_occurences'] = row['good_occurences'] + row['bad_occurences']

# loop through table and add good game containing word probability
for index, row in titles_table.iterrows():
    titles_table.loc[index,'good_game_containing_word_probability'] = row['good_occurences'] / len(good_df)
    
# #loop through table and add bad game containing word probability
for index, row in titles_table.iterrows():
    titles_table.loc[index,'bad_game_containing_word_probability'] = row['bad_occurences'] / len(bad_df)
    
# #loop through table and add good game given word probability
for index, row in titles_table.iterrows():
    titles_table.loc[index,'good_game_given_word_probability'] = row['good_occurences'] / row['total_occurences']
    
# #loop through table and add bad game given word probability
for index, row in titles_table.iterrows():
    titles_table.loc[index, 'bad_game_given_word_probability'] = row['bad_occurences'] / row['total_occurences']

titles_table.to_csv('titles_table.csv')
titles_table

,word,good_occurences,bad_occurences,total_occurences,good_game_containing_word_probability,bad_game_containing_word_probability,good_game_given_word_probability,bad_game_given_word_probability
0,flat,0,8,8,0.000000,0.000193,0.000000,1.000000
1,wolv,7,0,7,0.000547,0.000000,1.000000,0.000000
2,wet,0,7,7,0.000000,0.000169,0.000000,1.000000
3,look,7,14,21,0.000547,0.000339,0.333333,0.666667
4,mission,13,67,80,0.001016,0.001620,0.162500,0.837500
...,...,...,...,...,...,...,...,...
2183,singl,0,9,9,0.000000,0.000218,0.000000,1.000000
2184,creator,0,28,28,0.000000,0.000677,0.000000,1.000000
2185,drown,0,7,7,0.000000,0.000169,0.000000,1.000000
2186,breach,9,12,21,0.000703,0.000290,0.428571,0.571429


Working with Genres instead of Titles

In [85]:
bad_df = pd.read_csv('bad_games.csv')
bad_df = pd.DataFrame(bad_df)
bad_df

,Name,AppID,Short Description,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Review Text (First Review),Playtime Forever (First Review),Playtime Last Two Weeks (First Review),Playtime at Review (First Review),Timestamp Created (First Review),Timestamp Updated (First Review),Voted Up (First Review),Votes Up (First Review),Genres
0,18+ MEMORY,1222470,18+ MEMORY - is a classic memory game where yo...,5,Mixed,11,7,18,61.111111,[h1]At a Glance[/h1] [table] [tr] ...,66.0,0.0,17.0,2020-02-01 08:38:16,2020-02-01 08:38:16,0.0,38.0,"Casual, Indie"
1,Slotracers VR,1222480,Slotracers VR is a full slotracing experience ...,0,3 user reviews,2,1,3,66.666667,Played on HP Reverb G2 (WMR) without issue. T...,1521.0,0.0,79.0,2021-03-06 16:41:00,2021-03-06 16:41:00,1.0,10.0,"Casual, Indie, Racing, Simulation"
2,Gastova: The Witches of Arkana,1222540,Gastova is a lighthearted 2D platforming adven...,0,5 user reviews,5,0,5,100.000000,"Its a fun little adventure, not too hard, not ...",222.0,0.0,221.0,2023-08-10 01:45:34,2023-08-10 01:45:34,1.0,0.0,"Action, Adventure, Indie, RPG, Early Access"
3,Red Ronin,1222570,Red Ronin is a Turn-based but fast paced dash ...,6,Mostly Positive,38,10,48,79.166667,"""Turn Based But Fast Paced"" means, about 1/3 i...",191.0,0.0,191.0,2021-09-28 05:04:07,2021-09-28 05:15:02,0.0,40.0,"Action, Indie, Strategy"
4,Stocksynd House,1222600,Stocksynd House offers a narrative-driven VR e...,7,Positive,34,6,40,85.000000,Some quick notes as I just finished: +Very de...,80.0,0.0,80.0,2020-03-21 04:10:29,2020-03-21 04:10:29,1.0,17.0,"Adventure, Indie"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41353,Tamara In The Forsaken Dungeon,2686290,Tamara In The Forsaken Dungeon is a third-pers...,0,1 user reviews,1,0,1,100.000000,"Clearly an Indy game, but not too bad. Good F-...",67.0,67.0,20.0,2023-11-27 16:25:36,2023-11-27 16:25:36,1.0,3.0,"Adventure, Indie"
41354,Suika Shapes,2686690,Match a variety of colorful shapes in an ever-...,7,Positive,13,0,13,100.000000,This is the Definitive version of Suika. I am ...,376.0,376.0,66.0,2023-11-27 19:03:47,2023-11-27 19:03:47,1.0,9.0,"Action, Casual"
41355,SOS Zombie Survivors,2689600,SOS Zombie Survivors follows a group of surviv...,6,Mostly Positive,10,3,13,76.923077,The game combines traditional survival and wav...,365.0,365.0,302.0,2023-11-28 21:06:10,2023-11-28 21:06:10,1.0,9.0,Action
41356,24H Stories: The Blackout,2690290,"You are a young boy. Tonight, your parents hav...",0,1 user reviews,1,0,1,100.000000,"Great game from an indie developer, really kee...",98.0,98.0,96.0,2023-11-27 18:37:01,2023-11-27 18:37:01,1.0,0.0,Indie


In [86]:
#drop the columns we dont need
bad_df = bad_df.drop(columns=['AppID','Short Description','Review Text (First Review)','Playtime Forever (First Review)','Playtime Last Two Weeks (First Review)','Playtime at Review (First Review)','Timestamp Created (First Review)','Timestamp Updated (First Review)','Voted Up (First Review)','Votes Up (First Review)'])
bad_df

,Name,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Genres
0,18+ MEMORY,5,Mixed,11,7,18,61.111111,"Casual, Indie"
1,Slotracers VR,0,3 user reviews,2,1,3,66.666667,"Casual, Indie, Racing, Simulation"
2,Gastova: The Witches of Arkana,0,5 user reviews,5,0,5,100.000000,"Action, Adventure, Indie, RPG, Early Access"
3,Red Ronin,6,Mostly Positive,38,10,48,79.166667,"Action, Indie, Strategy"
4,Stocksynd House,7,Positive,34,6,40,85.000000,"Adventure, Indie"
...,...,...,...,...,...,...,...,...
41353,Tamara In The Forsaken Dungeon,0,1 user reviews,1,0,1,100.000000,"Adventure, Indie"
41354,Suika Shapes,7,Positive,13,0,13,100.000000,"Action, Casual"
41355,SOS Zombie Survivors,6,Mostly Positive,10,3,13,76.923077,Action
41356,24H Stories: The Blackout,0,1 user reviews,1,0,1,100.000000,Indie


In [115]:
#tokenize the genres and store in a list
genres = []
counter = 0
while counter < len(bad_df):
    genres.append(word_tokenize(str(bad_df.iloc[counter]['Genres'])))
    counter += 1
    
#flatten the list
genres = list(chain.from_iterable(genres))
#keep only the words
genres = [word for word in genres if word.isalpha()]
# get rid of non-printable characters
genres = [word for word in genres if word.isprintable()]
# get rid of emojis by looking for non-ascii characters
genres = [word for word in genres if word.encode('ascii', 'ignore').decode('ascii') == word]
# if an index contains early, and the next index contains access, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Early' and genres[counter + 1] == 'Access':
        genres[counter] = 'Early Access'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains free, and the next index contains to, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Free' and genres[counter + 1] == 'to':
        genres[counter] = 'Free to Play'
        genres.pop(counter+1)
    counter += 1

#if an index contains play, drop it
genres = [word for word in genres if word != 'Play']

#if an index contains massively, and the next index contains multiplayer, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Massively' and genres[counter + 1] == 'Multiplayer':
        genres[counter] = 'Massively Multiplayer'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains sexual, and the next index contains content, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Sexual' and genres[counter + 1] == 'Content':
        genres[counter] = 'Sexual Content'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains education, and the next index contains software, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Education' and genres[counter + 1] == 'Software':
        genres[counter] = 'Education Software'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains training, and the next index contains software, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Training' and genres[counter + 1] == 'Software':
        genres[counter] = 'Training Software'
        genres.pop(counter + 1)
    counter += 1
    

genres = pd.DataFrame(genres)
genres.to_csv('bad_genres_wordcloud_data.csv')
genres

,0
0,Casual
1,Indie
2,Casual
3,Indie
4,Racing
...,...
117665,Casual
117666,Action
117667,Indie
117668,Action


In [116]:
#create a vocabulary of the genres
bad_genres_vocab = list(set(genres[0]))

#turn genres into a list for the table
genres = genres[0].tolist()

bad_genres_vocab

['Education Software',
 'Massively Multiplayer',
 'Free to Play',
 'nan',
 'Racing',
 'Strategy',
 'Development',
 'Nudity',
 'Sexual Content',
 'Education',
 'Animation',
 'Gore',
 'Game',
 'Simulation',
 'Video',
 'Audio',
 'Adventure',
 'Publishing',
 'Akcja',
 'Violent',
 'Illustration',
 'Accounting',
 'Utilities',
 'RPG',
 'Modeling',
 'Software',
 'Casual',
 'Action',
 'Early Access',
 'Indie',
 'Editing',
 'Aventura',
 'Training Software',
 'Production',
 'Web',
 'Sports',
 'Photo',
 'Movie',
 'Training',
 'Design']

In [99]:
#count the number of times each genre appears in the vocab from genres
bad_genre_counts = []
for word in bad_genres_vocab:
    bad_genre_counts.append(genres.count(word))
bad_genre_counts = pd.DataFrame(bad_genre_counts)
bad_genre_counts = bad_genre_counts.rename(columns={0: 'count'})
bad_genre_counts['word'] = bad_genres_vocab
bad_genre_counts = bad_genre_counts[['word', 'count']]
bad_genre_counts = bad_genre_counts.sort_values(by=['count'], ascending=False)

#reset
bad_genre_counts = bad_genre_counts.reset_index(drop=True)
bad_genre_counts.to_csv('bad_genre_occurrences.csv')
bad_genre_counts

,word,count
0,Indie,30353
1,Casual,18563
2,Action,17604
3,Adventure,16241
4,Simulation,8045
5,Strategy,7557
6,RPG,6078
7,Early Access,4437
8,Sports,2073
9,Racing,1645


In [100]:
good_df = pd.read_csv('good_games.csv')
good_df = pd.DataFrame(good_df)
good_df

,Name,AppID,Short Description,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Review Text (First Review),Playtime Forever (First Review),Playtime Last Two Weeks (First Review),Playtime at Review (First Review),Timestamp Created (First Review),Timestamp Updated (First Review),Voted Up (First Review),Votes Up (First Review),Genres
0,Necromunda: Hired Gun,1222370,"Become a Hired Gun. The money’s good, the dog’...",6,Mostly Positive,2264,921,3185,71.083203,"A clunky arcade style fps. Its an unfinished, ...",71.0,0.0,71.0,2023-11-03 06:11:21,2023-11-03 06:29:40,0.0,15.0,"Action, Adventure"
1,The Soul Stone War,1222400,"Become the Wielder of Soul Stone, an ancient a...",6,Mostly Positive,55,22,77,71.428571,"First and Foremost, once again this book is no...",868.0,0.0,370.0,2020-03-06 06:56:40,2020-03-06 06:56:40,0.0,71.0,"Adventure, Casual, Indie, RPG"
2,A Way Out,1222700,A Way Out - это уникальное коллективное приклю...,8,Very Positive,10875,1564,12439,87.426642,we are not friends anymore but the story was i...,344.0,0.0,344.0,2023-11-11 20:19:51,2023-11-11 20:19:51,1.0,33.0,"Экшены, Приключенческие игры, Инди"
3,FutaDomWorld,1222750,"FutaDomWorld is a dystopian, futanari-femdom d...",8,Very Positive,59,4,63,93.650794,"You crafty LGBTQ+ futa bastards. You made me,...",928.0,0.0,768.0,2023-11-04 20:30:59,2023-11-04 20:30:59,1.0,5.0,"Casual, Indie, Early Access"
4,Umurangi Generation,1223500,Umurangi Generation is a first person photogra...,8,Very Positive,656,48,704,93.181818,"It may be pretentious, but ignore the time lim...",451.0,0.0,216.0,2023-09-08 16:21:07,2023-09-08 16:21:07,1.0,5.0,"Casual, Indie, Simulation"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12789,Filly Astray,2602780,A platforming adventure game with a new take o...,8,Very Positive,52,4,56,92.857143,"TLDR: 6/10. Good effort, extremely cute, but f...",40.0,0.0,40.0,2023-11-02 11:25:12,2023-11-02 11:28:02,1.0,52.0,Action
12790,Buried,2605640,You wake up buried alive. You are running out ...,8,Very Positive,59,14,73,80.821918,GAMEPLAY AND REVIEW https://www.youtube.com/w...,28.0,0.0,28.0,2023-11-16 09:02:07,2023-11-16 09:02:07,1.0,3.0,Indie
12791,Front Office Football Nine,2633170,Put yourself in the Front Office with Front Of...,6,Mostly Positive,40,12,52,76.923077,"Bought it within hours of it coming out, love ...",5566.0,2905.0,82.0,2023-11-02 14:21:02,2023-11-02 14:21:02,0.0,28.0,"Indie, Simulation, Sports"
12792,Lust Bunker [18+],2668170,Welcome to &quot;Lust Bunker&quot; – an enthra...,8,Very Positive,70,10,80,87.500000,The game literally has only 1 decision you can...,97.0,97.0,97.0,2023-11-18 14:02:31,2023-11-18 14:02:31,0.0,64.0,"Casual, RPG, Simulation"


In [101]:
#drop the columns we dont need
good_df = good_df.drop(columns=['AppID','Short Description','Review Text (First Review)','Playtime Forever (First Review)','Playtime Last Two Weeks (First Review)','Playtime at Review (First Review)','Timestamp Created (First Review)','Timestamp Updated (First Review)','Voted Up (First Review)','Votes Up (First Review)'])
good_df

,Name,Review Score,Review Score Description,Total Positive,Total Negative,Total Reviews,Positive Review Percentage,Genres
0,Necromunda: Hired Gun,6,Mostly Positive,2264,921,3185,71.083203,"Action, Adventure"
1,The Soul Stone War,6,Mostly Positive,55,22,77,71.428571,"Adventure, Casual, Indie, RPG"
2,A Way Out,8,Very Positive,10875,1564,12439,87.426642,"Экшены, Приключенческие игры, Инди"
3,FutaDomWorld,8,Very Positive,59,4,63,93.650794,"Casual, Indie, Early Access"
4,Umurangi Generation,8,Very Positive,656,48,704,93.181818,"Casual, Indie, Simulation"
...,...,...,...,...,...,...,...,...
12789,Filly Astray,8,Very Positive,52,4,56,92.857143,Action
12790,Buried,8,Very Positive,59,14,73,80.821918,Indie
12791,Front Office Football Nine,6,Mostly Positive,40,12,52,76.923077,"Indie, Simulation, Sports"
12792,Lust Bunker [18+],8,Very Positive,70,10,80,87.500000,"Casual, RPG, Simulation"


In [102]:
#tokenize the genres and store in a list
genres = []
counter = 0
while counter < len(good_df):
    genres.append(word_tokenize(str(good_df.iloc[counter]['Genres'])))
    counter += 1
    
#flatten the list
genres = list(chain.from_iterable(genres))
#keep only the words
genres = [word for word in genres if word.isalpha()]
# get rid of non-printable characters
genres = [word for word in genres if word.isprintable()]
# get rid of emojis by looking for non-ascii characters
genres = [word for word in genres if word.encode('ascii', 'ignore').decode('ascii') == word]
# if an index contains early, and the next index contains access, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Early' and genres[counter + 1] == 'Access':
        genres[counter] = 'Early Access'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains free, and the next index contains to, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Free' and genres[counter + 1] == 'to':
        genres[counter] = 'Free to Play'
        genres.pop(counter+1)
    counter += 1

#if an index contains play, drop it
genres = [word for word in genres if word != 'Play']

#if an index contains massively, and the next index contains multiplayer, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Massively' and genres[counter + 1] == 'Multiplayer':
        genres[counter] = 'Massively Multiplayer'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains sexual, and the next index contains content, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Sexual' and genres[counter + 1] == 'Content':
        genres[counter] = 'Sexual Content'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains education, and the next index contains software, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Education' and genres[counter + 1] == 'Software':
        genres[counter] = 'Education Software'
        genres.pop(counter + 1)
    counter += 1
    
#if an index contains training, and the next index contains software, combine them
counter = 0
while counter < len(genres):
    if genres[counter] == 'Training' and genres[counter + 1] == 'Software':
        genres[counter] = 'Training Software'
        genres.pop(counter + 1)
    counter += 1

genres = pd.DataFrame(genres)
genres.to_csv('good_genres_wordcloud_data.csv')
genres

,0
0,Action
1,Adventure
2,Adventure
3,Casual
4,Indie
...,...
34905,Simulation
34906,Casual
34907,Indie
34908,Simulation


In [103]:
#create a vocabulary of the genres
good_genre_vocab = list(set(genres[0]))
good_genre_vocab.sort()

genres = genres[0].tolist()

good_genre_vocab

['Acceso',
 'Accounting',
 'Action',
 'Adventure',
 'Aktion',
 'Animation',
 'Audio',
 'Aventura',
 'Azione',
 'Casual',
 'Design',
 'Development',
 'Early Access',
 'Editing',
 'Education',
 'Education Software',
 'Estrategia',
 'Free to Play',
 'Game',
 'Gore',
 'Illustration',
 'Indie',
 'Kostenlos',
 'Massively Multiplayer',
 'Modeling',
 'Multijugador',
 'Nudity',
 'Photo',
 'Production',
 'Publishing',
 'RPG',
 'Racing',
 'Rol',
 'Sexual Content',
 'Simuladores',
 'Simulation',
 'Software',
 'Sports',
 'Strategia',
 'Strategie',
 'Strategy',
 'Symulacje',
 'Training',
 'Utilities',
 'Video',
 'Violent',
 'Web',
 'anticipado',
 'masivo',
 'nan']

In [104]:
#count the number of times each genre appears in the vocab from genres
good_genre_counts = []
for word in good_genre_vocab:
    good_genre_counts.append(genres.count(word))
good_genre_counts = pd.DataFrame(good_genre_counts)
good_genre_counts['word'] = good_genre_vocab

#if one index is early and the next index is access, combine them
good_genre_counts.loc[good_genre_counts['word'] == 'Early', 'word'] = 'Early Access'
#remove the access
good_genre_counts = good_genre_counts[good_genre_counts['word'] != 'Access']
#reorder columns
good_genre_counts = good_genre_counts[['word', 0]]
#rename column
good_genre_counts = good_genre_counts.rename(columns={0: 'count'})
#sort by count
good_genre_counts = good_genre_counts.sort_values(by=['count'], ascending=False)
good_genre_counts.to_csv('good_genre_occurrences.csv')
good_genre_counts

,word,count
21,Indie,8318
2,Action,5545
3,Adventure,5382
9,Casual,3431
35,Simulation,3106
40,Strategy,2886
30,RPG,2838
12,Early Access,1051
37,Sports,564
31,Racing,442


In [114]:
#create table of the vocab
genres_table = []
#merge genre vocabularies into a set
genres_vocab = list(set(good_genre_vocab) | set(bad_genres_vocab))
genres_table = pd.DataFrame(genres_vocab)
#rename column to word
genres_table = genres_table.rename(columns={0: 'word'})
#add column for good occurences
genres_table['good_occurences'] = 0
#add column for bad occurences
genres_table['bad_occurences'] = 0
#add column for total occurences
genres_table['total_occurences'] = 0
#add column for good game containing word probability
genres_table['good_game_containing_word_probability'] = 0
#add column for bad game containing word probability
genres_table['bad_game_containing_word_probability'] = 0
#add column for good game given word probability
genres_table['good_game_given_word_probability'] = 0
#add column for bad game given word probability
genres_table['bad_game_given_word_probability'] = 0

#map good occurences to table
genres_table = genres_table.merge(good_genre_counts, on='word', how='left')
#map bad occurences to table
genres_table = genres_table.merge(bad_genre_counts, on='word', how='left')
#fill NaN values in count_x and count_y with 0
genres_table = genres_table.fillna(0)
#turn count_x and count_y into integers
genres_table['good_occurences'] = genres_table['count_x'].astype(int)
genres_table['bad_occurences'] = genres_table['count_y'].astype(int)
#drop count_x and count_y columns
genres_table = genres_table.drop(columns=['count_x', 'count_y'])
#drop indexes
x = ['Kostenlos','nan','Symulacje','Simuladores','Akcja','massivo','Aktion','Aventura','Strategia','Azione','Multijugador','Rol','Estrategia','Accesso','anticipado']
genres_table = genres_table[~genres_table['word'].isin(x)]

    
# loop through table and add total occurences
for index, row in genres_table.iterrows():
    genres_table.loc[index, 'total_occurences'] = row['good_occurences'] + row['bad_occurences']

# loop through table and add good game containing word probability
for index, row in genres_table.iterrows():
    genres_table.loc[index,'good_game_containing_word_probability'] = row['good_occurences'] / len(good_df)
    
# #loop through table and add bad game containing word probability
for index, row in genres_table.iterrows():
    genres_table.loc[index,'bad_game_containing_word_probability'] = row['bad_occurences'] / len(bad_df)
    
# #loop through table and add good game given word probability
for index, row in genres_table.iterrows():
    genres_table.loc[index,'good_game_given_word_probability'] = row['good_occurences'] / row['total_occurences']
    
# #loop through table and add bad game given word probability
for index, row in genres_table.iterrows():
    genres_table.loc[index, 'bad_game_given_word_probability'] = row['bad_occurences'] / row['total_occurences']

#sort by good_game_containing_word_probability
genres_table = genres_table.sort_values(by=['good_game_containing_word_probability'], ascending=False)

# genres_table.to_csv('genres_table.csv')
genres_table

,word,good_occurences,bad_occurences,total_occurences,good_game_containing_word_probability,bad_game_containing_word_probability,good_game_given_word_probability,bad_game_given_word_probability
21,Indie,8318,30353,38671,0.650149,0.733909,0.215097,0.784903
45,Action,5545,17604,23149,0.433406,0.425649,0.239535,0.760465
39,Adventure,5382,16241,21623,0.420666,0.392693,0.248902,0.751098
44,Casual,3431,18563,21994,0.268173,0.448837,0.155997,0.844003
36,Simulation,3106,8045,11151,0.242770,0.194521,0.278540,0.721460
6,Strategy,2886,7557,10443,0.225574,0.182722,0.276357,0.723643
17,RPG,2838,6078,8916,0.221823,0.146961,0.318304,0.681696
46,Early Access,1051,4437,5488,0.082148,0.107283,0.191509,0.808491
27,Sports,564,2073,2637,0.044083,0.050123,0.213879,0.786121
5,Racing,442,1645,2087,0.034547,0.039775,0.211787,0.788213
